In [76]:
from tkinter import *
from os.path import exists
import pandas as pd
from odbm.odbm_main import ModelBuilder, ModelHandler
from odbm.utils import extractParams, fmt


In [62]:
# Checks if model path given by user exists and builds model.
def loadModel(model_path,root):
    global myModel #create global variable so it is accessible in main
    if exists(model_path) and model_path.endswith(('.xls','.xlsx')):
        model_species = pd.read_excel(model_path, sheet_name = 'Species & Base Mechanisms', engine = 'openpyxl').dropna('index','all')
        model_rxns = pd.read_excel(model_path, sheet_name = 'Reaction', engine = 'openpyxl').dropna('index','all')
        myModel = ModelBuilder(model_species, model_rxns)
        print('Model successfully built')
        modelWindow(myModel, model_path, root)

    else:
        if exists(model_path):
            raise TypeError('Please pass in an excel compatible workbook with sheets named "Species & Base Mechanisms" and "Reaction"')
        else:
            raise FileExistsError('Error: No file exists with given name.')

In [133]:
# Main Window
def __init__():
    root = Tk()
    frame = Frame(root)
    frame.pack()

    welcome = Label(frame, text = "Welcome to ODBM \n Please insert file path for model to load:")
    welcome.config(font = ('Arial', 12))
    entry = Entry(frame, width = 40)
    loadbutton = Button(frame, text="Load Model", fg="blue", command = lambda:loadModel(entry.get(),root))

    welcome.pack()
    loadbutton.pack(side = BOTTOM)
    entry.pack()
    root.mainloop()

__init__()

Model successfully built


In [126]:
# 2nd Window
def modelWindow(myModel, model_path, root):

    frame, root = newWindow(root)

    welcome = Label(frame, text = "Model successfully loaded! ")
    welcome.config(font = ('Arial', 12))
    welcome.pack()
    
    cancel_button = Button(frame, text="Cancel", fg="blue",command = lambda: cancelButton(root)) 
    save_button = Button(frame, text = 'Save Model', fg = 'blue', command = lambda: saveModelTxt(myModel,model_path,frame))
    simulate_button = Button(frame, text = 'Simulate Model', fg = 'blue', command = lambda: runSimulation(myModel, model_path, root)) 
    edit_button = Button(frame, text = 'Edit Model', fg = 'blue') # need a command to open edit window

    cancel_button.pack(side = LEFT)
    save_button.pack(side = LEFT)
    simulate_button.pack(side = LEFT)
    edit_button.pack(side = LEFT)


In [129]:
def cancelButton(root):
    root.destroy()
    __init__()

def newWindow(root):
    root.destroy()
    root = Tk()
    frame = Frame(root)
    frame.pack()
    return frame, root

def saveModelTxt(myModel,model_path,frame):
    modelfile = model_path[:-4] + 'txt'
    myModel.saveModel(modelfile)
    save_msg = Label(frame, text = 'Model successfully saved.')
    save_msg.pack(side = BOTTOM)
    return modelfile


def runSimulation(myModel, model_path, root):

    # Close previous window and open new
    frame, root = newWindow(root)

    # Set up ModelHandler instance and selections for simulation
    modelfile = saveModelTxt(myModel,model_path, frame)
    with open(modelfile,'r') as m:
        MH = ModelHandler(m.read())
    selections = list(map(fmt, myModel.species['Label'])) + list(myModel.rxns['Label']) + ['time']


    #Add entries, buttons, labels
    start = Entry(frame, width = 10)
    end = Entry(frame, width = 10)
    points = Entry(frame, width = 10)
    titles = Label(frame, text = "Start: \t End: \t Points: \n")
    run_button = Button(frame, text = 'Run Simulation',  fg = 'blue', 
    command = lambda : [MH.setSimParams(int(start.get()),int(end.get()),int(points.get()), selections=selections), 
                        postSimulation(MH, model_path, root)])
    # Does sensitivity analysis run simulation even if we don't want to do sensitivity analysis? 
    # It is the only place in odbm_main that calls simulate
    # Right now calling sensitivityAnalysis in "postSimulation" 
    # even though that is technically where simulation is happening because want to be able to save output

    titles.pack(side = LEFT)
    start.pack(side = LEFT)
    end.pack(side = LEFT)
    points.pack(side = LEFT)
    run_button.pack(side = RIGHT)

def postSimulation(MH, model_path, root):
    # Close previous window and open new
    frame, root = newWindow(root)

    results = MH.sensitivityAnalysis()

    #Add entries, buttons, labels
    back = Button(frame, text = 'Back',  fg = 'blue', command = lambda: runSimulation(myModel,model_path,root)))
    save = Button(frame,text = "Save Simulation Results", fg = 'blue') #add function to save sim results to excel file
    visualize = Button(frame, text = 'Visualize Simulation Results', fg = 'blue') #use vis.py, let user input reactions and/or species


    


In [ ]:
C:\Users\rycar\Documents\GitHub\ODBM\Examples\model_definition.xlsx

In [120]:
myModel.rxns

,Label,Enzyme,Mechanism,Substrate,Cofactor,Product,Parameters,Unnamed: 7,Unnamed: 8
0,R1,NaN,MA,"A,B",NaN,C,k:10,NaN,NaN
1,R2,NaN,MA,C,NaN,NaN,k:.1,NaN,NaN
2,R3,NaN,MA,B,NaN,2A,k:2,NaN,NaN


In [75]:
myModel.get_products(id = 'R1')

AttributeError: 'Series' object has no attribute 'split'